# Sensors Analytics REST API

## Deploy the API
- Go to the *Machine* tab, then set *Incoming connections* to **ON**. The API will be accessible through the indicated tunnelling link.  
- Run the notebook.



## Import the required modules

In [1]:
import cherrypy
import json
import redis

## Connect to the Redis Database

In [2]:
import redis

REDIS_HOST = 'redis-11392.c300.eu-central-1-1.ec2.redns.redis-cloud.com'
REDIS_PORT = 11392
REDIS_USERNAME = 'default'
REDIS_PASSWORD = 'nVukRxv3hvJTuYLkK6n4XdWF8etIzoOO'

redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)

is_connected = redis_client.ping()
print('Redis Connected:', is_connected)

Redis Connected: True


### Status Endpoint Class

In [3]:
class Status(object):
    exposed = True

    def GET(self, *path, **query):
        response_dict = {
            'status': 'online'
        }
        response = json.dumps(response_dict)

        return response

## Sensors Endpoint Class

In [4]:
class Sensors(object):
    exposed = True

    def GET(self, *path, **query):
        # print(query)
        min_t_samples = int(query.get('min_t_samples', 0))
        min_h_samples = int(query.get('min_h_samples', 0))
        sensors = []
        keys = redis_client.keys('0x*:temperature')

        count = 0
        for key in keys:
            key = key.decode()
            mac_address = key.split(':')[0]

            t_info = redis_client.ts().info(f'{mac_address}:temperature')
            t_samples = t_info.total_samples
            t_retention = t_info.retention_msecs
            h_info = redis_client.ts().info(f'{mac_address}:humidity')
            h_samples = h_info.total_samples
            h_retention = h_info.retention_msecs
            
            if t_samples >= min_t_samples and h_samples >= min_h_samples:
                sensors.append(
                    {
                        "mac_address": mac_address,
                        "t_samples": t_samples,
                        "t_retention": t_retention,
                        "h_samples": h_samples,
                        "h_retention": h_retention,
                    }
                )
                count += 1

        response_dict = {
            "sensors": sensors,
            "count": count,
        }

        response = json.dumps(response_dict)

        return response

    def POST(self, *path, **query):
        body = cherrypy.request.body.read()
        # print(body)
        body_dict = json.loads(body.decode())
        # print(body_dict)

        mac_address = body_dict.get('mac_address', None)

        if mac_address is None:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request body.')

        try:
            redis_client.ts().create(f'{mac_address}:temperature', retention_msecs=24*60*60*1000)
        except redis.ResponseError:
            raise cherrypy.HTTPError(409, 'Sensor already exists.')

        try:
            redis_client.ts().create(f'{mac_address}:humidity', retention_msecs=24*60*60*1000)
        except redis.ResponseError:
            raise cherrypy.HTTPError(409, 'Sensor already exists.')

        return

## Sensor Endpoint Class

In [5]:
class Sensor(object):
    exposed = True

    def GET(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request parameters.')

        mac_address = path[0]

        try:
            t_info = redis_client.ts().info(f'{mac_address}:temperature')
        except redis.ResponseError:
            raise cherrypy.HTTPError(404, 'MAC address not found in the database.')
        
        h_info = redis_client.ts().info(f'{mac_address}:humidity')

        response_dict = {
            "mac_address": mac_address,
            "t_samples": t_info.total_samples,
            "t_retention": t_info.retention_msecs,
            "h_samples": h_info.total_samples,
            "h_retention": h_info.retention_msecs,
        }

        response = json.dumps(response_dict)

        return response

    def PUT(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request parameters.')

        mac_address = path[0]

        try:
            t_info = redis_client.ts().info(f'{mac_address}:temperature')
        except redis.ResponseError:
            raise cherrypy.HTTPError(404, 'MAC address not found in the database.')

        body = cherrypy.request.body.read()
        # print(body)
        body_dict = json.loads(body.decode())
        # print(body_dict)

        t_retention = body_dict.get('t_retention', None)

        if t_retention is None:
            raise cherrypy.HTTPError(400, 'Missing temperature retention period in the request body.')

        h_retention = body_dict.get('h_retention', None)

        if h_retention is None:
            raise cherrypy.HTTPError(400, 'Missing humidity retention period in the request body.')

        redis_client.ts().alter(f'{mac_address}:temperature', retention_msecs=t_retention)
        redis_client.ts().alter(f'{mac_address}:humidity', retention_msecs=h_retention)

        return
    
    def DELETE(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request parameters.')
        
        mac_address = path[0]
        found = 0
        found += redis_client.delete(f'{mac_address}:temperature')
        found += redis_client.delete(f'{mac_address}:humidity')

        if found == 0:
            raise cherrypy.HTTPError(404, 'MAC address not found in the database.')

        return

## Data Endpoint Class

In [6]:

import datetime
from dateutil.parser import parse as parse_date

class Data(object):
    exposed = True

    
    def GET(self, *path, **query):

        #path paramters: mac_address
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request parameters.')

        mac_address = path[0]

        # Check for required query parameters
        start_date = query.get('start_date')
        end_date = query.get('end_date')

        if not start_date:
            raise cherrypy.HTTPError(400, 'Missing start date in the request parameters.')

        if not end_date:
            raise cherrypy.HTTPError(400, 'Missing end date in the request parameters.')

        # Validate date format
        try:
            start_date_parsed = parse_date(start_date)
        except ValueError:
            raise cherrypy.HTTPError(400, 'Wrong format for start date in the request parameters.')

        try:
            end_date_parsed = parse_date(end_date)
        except ValueError:
            raise cherrypy.HTTPError(400, 'Wrong format for end date in the request parameters.')

        # Check date range validity
        if end_date_parsed <= start_date_parsed:
            raise cherrypy.HTTPError(400, 'End date smaller or equal than start date.')

        # Convert date to timestamps in ms
        start_timestamp = int(start_date_parsed.timestamp() * 1000)
        end_timestamp = int(end_date_parsed.timestamp() * 1000)

        # Attempt to fetch the data from Redis
        try:
            #timestamps = redis_client.ts().range(f'{mac_address}:timestamp', start_timestamp, end_timestamp)
            temperatures = redis_client.ts().range(f'{mac_address}:temperature', start_timestamp, end_timestamp)
            humidities = redis_client.ts().range(f'{mac_address}:humidity', start_timestamp, end_timestamp)
        except redis.ResponseError:
            raise cherrypy.HTTPError(404, 'MAC address not found in the database.')

        # Prepare the response
        response_dict = {
            "mac_address": mac_address , 
            "timestamp": [int(t[0]) for t in temperatures],  # Use temperatures for timestamps
            "temperature": [int(t[1]) for t in temperatures],
            "humidity": [int(h[1]) for h in humidities],
        }

        response = json.dumps(response_dict)
        return response



## Setup cherrypy and Map objects to their target endpoints

In [7]:
if __name__ == '__main__':
    conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}}
    cherrypy.tree.mount(Data(), '/data', conf)
    cherrypy.tree.mount(Status(), '/status', conf)
    cherrypy.tree.mount(Sensors(), '/sensors', conf)
    cherrypy.tree.mount(Sensor(), '/sensor', conf)
    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080})
    cherrypy.engine.start()
    cherrypy.engine.block()

[31/Dec/2024:16:19:44] ENGINE Bus STARTING
[31/Dec/2024:16:19:44] ENGINE Started monitor thread 'Autoreloader'.
[31/Dec/2024:16:19:44] ENGINE Serving on http://0.0.0.0:8080
[31/Dec/2024:16:19:44] ENGINE Bus STARTED
172.3.28.49 - - [31/Dec/2024:16:19:47] "GET /status HTTP/1.1" 200 20 "" "python-requests/2.32.3"
172.3.28.49 - - [31/Dec/2024:16:21:47] "GET /data/0xe45f01e89bcc?start_date=2024-12-29&end_date=2024-12-30 HTTP/1.1" 200 1014 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
172.3.186.144 - - [31/Dec/2024:16:24:40] "POST /sensors HTTP/1.1" 200 - "" "python-requests/2.32.3"
172.3.50.42 - - [31/Dec/2024:16:27:38] "GET /data/0xe45f01e89bcc?start_date=2024-12-29&end_date=2024-12-30 HTTP/1.1" 200 1014 "" "python-requests/2.32.3"
[31/Dec/2024:21:59:13] ENGINE Keyboard Interrupt: shutting down bus
[31/Dec/2024:21:59:13] ENGINE Bus STOPPING
[31/Dec/2024:21:59:13] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer((

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8a9d9526-dc21-42d6-ba37-8f708634743d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>